In [2]:
# Импортируем необходимые библиотеки
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Настраиваем начальные параметры
IMG_HEIGHT, IMG_WIDTH = 64, 192  # Высота и ширина изображений
BATCH_SIZE = 32
EPOCHS = 50

In [ ]:
# Шаг 1: Подготовка данных
# папка 'data' с изображениями и файл 'labels.csv' с метками

data_dir = 'data/images'  # Путь к изображениям
labels_file = 'data/labels.csv'  # Путь к файлу с метками

# Загружаем метки из CSV файла
labels_df = pd.read_csv(labels_file)
labels = labels_df['label'].values  # Предполагаем, что колонка с метками называется 'label'

# Загружаем и обрабатываем изображения
def load_images(data_dir, labels):
    images = []
    for label in labels:
        image_path = os.path.join(data_dir, label)  # Получаем путь к изображению
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Конвертируем в градации серого
        image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))  # Изменяем размер
        images.append(image)
    return np.array(images)

images = load_images(data_dir, labels)

# Нормализуем изображения для лучшей сходимости
images = images.astype('float32') / 255.0

# Шаг 2: Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Шаг 3: Создание модели
model = keras.Sequential([
    layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 1)),  # Входной слой
    layers.Conv2D(32, (3, 3), activation='relu'),  # Сверточный слой
    layers.MaxPooling2D(pool_size=(2, 2)),  # Подвыборка
    layers.Conv2D(64, (3, 3), activation='relu'),  # Еще один сверточный слой
    layers.MaxPooling2D(pool_size=(2, 2)),  # Подвыборка
    layers.Conv2D(128, (3, 3), activation='relu'),  # Третий сверточный слой
    layers.MaxPooling2D(pool_size=(2, 2)),  # Подвыборка
    layers.Flatten(),  # Сглаживание для входа в Dense слои
    layers.Dense(128, activation='relu'),  # Полносвязный слой
    layers.Dense(len(set(labels)), activation='softmax')  # Выходной слой с функцией активации softmax
])

# Шаг 4: Компиляция модели
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Используем crossentropy для классификации
              metrics=['accuracy'])

# Шаг 5: Обучение модели
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)

# Шаг 6: Оценка модели
test_loss, test_acc = model.evaluate(X_test, y_test)
print("\nTest accuracy:", test_acc)

# Шаг 7: Визуализация результатов
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Шаг 8: Сохранение модели (по желанию)
model.save('license_plate_recognition_model.h5')